In [32]:
import numpy as np
import cv2
import math

In [33]:
webcam = cv2.VideoCapture(0)

while webcam.isOpened():
    
    ret, image = webcam.read()
    
    cv2.rectangle(image, (100,100),(300,300),(0,255,0),0)
    crop_image = image[100:300, 100:300]
    
    gaussian = cv2.GaussianBlur(crop_image, (3,3),0)
    
    hsv = cv2.cvtColor(gaussian, cv2.COLOR_BGR2HSV)
    
    mask1= cv2.inRange(hsv, np.array([2,0,0]), np.array([20,255,255]))
    
    kernel = np.ones((5,5))
    
    dilation = cv2.dilate(mask1, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)
    
    filtered = cv2.GaussianBlur(erosion, (3,3), 0)
    ret, thresh = cv2.threshold(filtered, 127, 255, 0)
    #thresh = cv2.flip(thresh, 1)
    cv2.imshow("Thresholed", thresh)
    
    temp = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #print(temp[1])
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    
    
    try :
        # On detecte le contour le plus grand, celui qui a le plus de chance d'être le contour de la main
        contour = max(contours, key = lambda x: cv2.contourArea(x))
        
        # On recupere un rectacle qui englobe toute notre structure (x,y) coordonnees 
        # et (w,h) width et heigh du rectangle
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image, (x,y),(x+w,h+y),(0,0,255),0)
        
        # On englobe le contour de la main dans une enveloppe convexe composée de segments
        hull = cv2.convexHull(contour)
        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1,(0,255,0),0)
        cv2.drawContours(drawing, [hull], -1,(0,255,0),0)
        
        # Detecter la difference avec returnPoints
        hull = cv2.convexHull(contour, returnPoints = False)
        
        # On recupere les points qui caracterisent cette enveloppe pour notre contour
        defects = cv2.convexityDefects(contour, hull)
        count_defects = 0
        
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            
            # start : debut du segment
            start =tuple(contour[s][0])
            # end : fin du segment
            end =tuple(contour[e][0])
            # far : point du contour le plus eloigné du milieu du segment
            far =tuple(contour[f][0])
            
            # longeur du segment
            a = math.sqrt((end[0]-start[0])**2 +(end[1]-start[1])**2 )
            #distance entre le point le plus eloigne et a
            b = math.sqrt((far[0]-start[0])**2 +(far[1]-start[1])**2 )
            #distance entre le point le plus eloigne et b
            c = math.sqrt((far[0]-end[0])**2 +(far[1]-end[1])**2 )
            
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c))
            if angle <=90 and b>=20:
                count_defects+=1
                cv2.circle(crop_image, far, 1,[0,0,255],-1)
            cv2.line(crop_image,start,end,[0,255,0],2)
            
        print(count_defects)
        if count_defects ==0:
            cv2.putText(image, "HELLO", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
        if count_defects ==1:
            cv2.putText(image, "TWO", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
        if count_defects ==2:
            cv2.putText(image, "THREE", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
        if count_defects ==3:
            cv2.putText(image, "FOUR", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
        if count_defects ==4:
            cv2.putText(image, "FIVE", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
        else :
            cv2.putText(image, " ", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
            pass
            
        """"""
    except :
        print('pass')
        pass
    
    #cv2.imshow("test hsv" ,image)
    # cv2.imshow("test hsv" ,hsv)

    if cv2.waitKey(1) == ord('a'):
        break
    
    image = cv2.flip(image, 1)

    cv2.imshow("Gesture", image)
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow("Contours", all_image)
    
webcam.release()
cv2.destroyAllWindows()

2
1
2
3
2
2
1
1
4
2
1
2
1
1
3
2
2
1
2
2
2
1


KeyboardInterrupt: 

: 